In [1]:
import pickle
from os.path import join
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import numpy as np
import pandas as pd

In [2]:
result_dir = '/bsuscratch/zacharykeskinen/data/infrasound/wind_results'
fig_dir = '/bsuhome/zacharykeskinen/infrasound/figures'
tmp_dir = join(result_dir,'tmp')
super_x = []
for f in glob(join(tmp_dir, '*')):
    with open(f, 'rb') as f:
        df = pickle.load(f)
    super_x.append(df)
res = pd.concat(super_x)
res['wind_cat'] = pd.cut(res['wind'], bins = [-0.1,3,5,11], labels = ['low-wind','med-wind','high-wind'])
with open(join(result_dir, 'windv2.pkl'), 'wb') as f:
    pickle.dump(res, f)


ValueError: No objects to concatenate

In [ ]:
result_dir = '/bsuscratch/zacharykeskinen/data/infrasound/wind_results'
with open(join(result_dir, 'windv3.pkl'), 'rb') as f:
    wind = pickle.load(f)
wind['wind_cat'] = pd.cut(wind['wind'], bins = [-0.1,3,5,11], labels = ['Low Wind (0-3 m/s)','Med Wind (3-5 m/s)','High Wind (5+ m/s)'])

: 

In [ ]:
wind[['fs','powers']] = pd.DataFrame(wind['power'].tolist(), index=wind.index)
# wind = wind.loc[wind.cor < wind.cor.quantile(0.8)]
# wind = wind.loc[wind.wind > 6]
# wind = wind.drop(['power'], axis = 1)
powers = pd.DataFrame(wind['powers'].tolist(), index=wind.index, columns = wind.iloc[0]['fs'])
powers[['sd_delta','cor','wind']] = wind[['sd_delta','cor','wind']]
#powers.loc[powers.sd_delta < -0.5, 'sd_delta'] = -0.5
plot = powers.drop(['cor','wind'], axis= 1).groupby('sd_delta').mean()
new_index = np.linspace(plot.index[0], plot.index[-1], len(plot))
df_resampled = plot.reindex(plot.index.union(new_index)).interpolate(method='nearest').loc[new_index]

: 

In [ ]:
levels = [[None,0],[0,0.2],[0.2,0.4],[0.4,0.6],[0.6,0.8],[0.8,1],[1,1.2],[1.2,None]]
plot = pd.DataFrame()
for low, high in levels:
    d = {}
    if low == None:
        sub = powers.loc[powers.sd_delta < high]
        low = -0.2
    elif high == None:
        sub = powers.loc[powers.sd_delta > low]
    else:
        sub = powers.loc[(powers.sd_delta > low) & (powers.sd_delta < high)]
    sub = sub.drop(['sd_delta', 'cor', 'wind'], axis = 1).dropna().mean()
    sub.index = np.round(sub.index.astype(float),2)
    d[low] = sub
    plot = pd.concat([plot, pd.DataFrame.from_dict(d)], axis = 1)

: 

In [ ]:
from re import A


f, axes = plt.subplots(3, figsize= (12,8))
levels = [[None,0],[0,0.2],[0.2,0.4],[0.4,0.6],[0.6,0.8],[0.8,1],[1,1.2],[1.2,None]]

for i, (low, high) in enumerate([[0,3],[3,5],[5,10]]):
    ax = axes[i]
    power_sub = powers.loc[(powers.wind < high) & (powers.wind > low)]
    plot = pd.DataFrame()
    for low, high in levels:
        d = {}
        if low == None:
            sub = power_sub.loc[power_sub.sd_delta < high]
            low = -0.2
        elif high == None:
            sub = power_sub.loc[power_sub.sd_delta > low]
        else:
            sub = power_sub.loc[(power_sub.sd_delta > low) & (power_sub.sd_delta < high)]
        sub = sub.drop(['sd_delta', 'cor', 'wind'], axis = 1).dropna().mean()
        sub.index = np.round(sub.index.astype(float),2)
        d[low] = sub
        plot = pd.concat([plot, pd.DataFrame.from_dict(d)], axis = 1)
    im = ax.imshow(plot[::-1], aspect = 'auto',vmin=-0.0000005, vmax=0.000003, interpolation = 'bilinear',extent = (df_resampled.index[0], df_resampled.index[-1], df_resampled.columns[0], df_resampled.columns[-1]))
    plt.colorbar(im, ax= ax)

: 

In [ ]:
from matplotlib.colors import LogNorm
f, axes = plt.subplots(3, figsize= (12,8))
norm=LogNorm(vmin=1e-7, vmax=2e-5)
title_dic = {'0-3':'Low Wind Speeds', '3-5':'Medium Wind Speeds', '5-10':'High Wind Speeds'}
for i, (low, high) in enumerate([[0,3],[3,5],[5,10]]):
    ax = axes[i]
    with open(join(result_dir, 'windv2.pkl'), 'rb') as f:
        wind = pickle.load(f)
    wind[['fs','powers']] = pd.DataFrame(wind['power'].tolist(), index=wind.index)
    # wind = wind.loc[wind.cor < wind.cor.quantile(0.5)]
    wind.loc[wind.sd_delta < -0.2, 'sd_delta'] = -0.2
    powers = pd.DataFrame(wind['powers'].tolist(), index=wind.index, columns = wind.iloc[0]['fs'])
    powers[['sd_delta','cor','wind']] = wind[['sd_delta','cor','wind']]

    plot = powers.loc[(powers.wind < high) & (powers.wind > low)].drop(['cor','wind'], axis= 1).groupby('sd_delta').mean()
    new_index = np.linspace(plot.index[0], plot.index[-1], len(plot))
    #df_resampled = plot.reindex(plot.index.union(new_index)).interpolate(method='nearest').loc[new_index]
    df_resampled = plot.reindex(np.arange(-0.2, 1, 0.1), method = 'nearest')
    extent = (df_resampled.index[0], df_resampled.index[-1], df_resampled.columns[0], df_resampled.columns[-1])
    # extent = (-0.2,1,0,100)
    im = ax.imshow(df_resampled.T.iloc[::-1],extent = extent, aspect = 'auto', norm = norm, interpolation = None)
    ax.set_ylabel('Frequency (Hz)')
    title = title_dic[f'{low}-{high}']
    # ax.set_title(f'Wind Speeds from {low} to {high} m/s')
    ax.set_title(title)
    plt.colorbar(im, ax = ax)
axes[2].set_xlabel('Sensor Burial Depth (m)')
#plt.xlim(-0.2,0.5)
plt.tight_layout()

: 

## scatter plots

In [ ]:
plt.subplots(figsize = (12,8))
plt.scatter(wind[(wind.sd_delta < -0.1) & (wind.cor > 0)].wind, wind[(wind.sd_delta < -0.1) & (wind.cor > 0)].broad_power)
plt.yscale('log')
plt.ylabel('Hourly Power')
plt.xlabel('Hourly Average Wind Speed')
plt.title('Canyon Creek Hourly Wind Speed vs Banner "Free-Air" Power')
plt.savefig(join(fig_dir, 'wind','free-air-canyon-scatter.png'))

: 

In [ ]:

sns.catplot(data = wind[(wind.h != 2) & (wind.sd_delta < -0.1)], x = 'wind_cat', y = 'broad_power', kind = 'box', height=8.27, aspect=11.7/8.27)
plt.yscale('log')
plt.ylabel('Banner Free-Air Power')
plt.xlabel('')
plt.xticks(ticks = [0,1,2], labels = ['Low Wind','Medium Wind','High Wind'])
plt.title('Canyon Wind Speeds vs Free-Air Banner Power')
plt.savefig(join(fig_dir, 'wind','free-air-canyon-box.png'))

: 

In [ ]:
wind.wind_cat.unique()

: 

In [ ]:
from sklearn.linear_model import LinearRegression
plt.subplots(figsize = (12,8))
for i, (low, high) in enumerate([[0,3],[5,10]]):
    # if i == 2:
        plot = wind.loc[(wind.wind < high) & (wind.wind > low) & (wind.cor > 0.2)]
        plt.scatter(plot.sd_delta, plot.broad_power, label = f'{low}-{high} m/s wind speed', alpha = 0.4)
        reg = LinearRegression().fit(plot.sd_delta.values.reshape(-1, 1), plot.broad_power.values.reshape(-1, 1))
        x = np.linspace(-0.5, 1.7, 1000)
        y = reg.predict(x.reshape(-1, 1))
        plt.plot(x,y)

plt.legend()
plt.yscale('log')
plt.ylim(10e0, 8e3)
plt.ylabel('Hourly Power')
plt.xlabel('Burial Depth')
plt.title('Hourly Power vs Burial Depth')
plt.savefig(join(fig_dir, 'wind','scatter_power_depth.png'))

: 

In [ ]:
import seaborn as sns
sns.set(rc={
 'axes.axisbelow': True,
 'axes.edgecolor': 'grey',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.grid.axis':'y',
 'axes.labelcolor': 'black',
 'axes.linewidth': 1,
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'figure.figsize': (18, 12),
 'grid.color': 'grey',
 'grid.alpha':1,
 'grid.linewidth':0.5,
 'image.aspect':'auto',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'savefig.dpi':300,
 'text.color': 'black',
 'xtick.bottom': False,
 'xtick.color': 'grey',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'grey',
 'ytick.direction': 'out',
 'ytick.left': True,
 'ytick.color':'grey',
 'ytick.right': False})
sns.set_context("notebook", rc={"font.size":20,
                                "axes.titlesize":30,
                                "axes.labelsize":24,
                                "axes.legendsize":18})

: 

## Paper Figure of Wind Speed

In [ ]:
f, axes = plt.subplots(3)
wind_snow = wind.loc[(wind.h != 2.00) & (wind.cor < 1)].copy()
wind_snow.loc[:, 'broad_power'] = wind_snow.loc[:, 'broad_power']/(60*60*200)
ylabels = ['Above Snow','0- 20 Centimeters','20+ Centimeters']
for j, h in enumerate([0, (0, 0.2), 0.2]):
        if j == 0:
            plot = wind_snow.loc[(wind_snow.sd_delta < h)]
        if j == 1:
            ax = plot = wind_snow.loc[(wind_snow.sd_delta > h[0]) & (wind_snow.sd_delta < h[1])]
        if j == 2:
            plot = wind_snow.loc[(wind_snow.sd_delta > h)]
        
        # plot.loc[:,'broad_power'] = (plot['broad_power']/(60*60)).values
        ax = sns.boxplot(y = 'wind_cat', x = 'broad_power', data = plot, ax = axes[j], showcaps = False, flierprops={"marker": "x"}, fliersize= 2, width = 0.4)

        ax.set_ylabel(ylabels[j])
        if j == 0:
            ax.set_title('Burial Depth vs Hourly Power')
        if j == 2:
            ax.set_xlabel('Hourly Power ($Pa^{2}$)')
        else:
            ax.set_xlabel('')
            ax.set_xticks([])
        ax.set_xscale('log')
        ax.set_xlim(10/(60*60*200), 10e5/(60*60*200))
plt.savefig(join('/bsuhome/zacharykeskinen/infrasound/figures/wind','boxplot_wind_depth_binv4'), dpi = 300)

: 

## P Test

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(wind.loc[wind.wind_cat == 'med-wind', 'broad_power'].values, wind.loc[wind.wind_cat == 'low-wind', 'broad_power'].values)

: 

In [ ]:
from scipy.stats import normaltest
normaltest(wind.loc[wind.wind_cat == 'med-wind', 'broad_power'].values)

: 

In [ ]:
from scipy.stats import mannwhitneyu
mannwhitneyu(wind.loc[wind.wind_cat == 'med-wind', 'broad_power'].values, wind.loc[wind.wind_cat == 'low-wind', 'broad_power'].values)

: 

In [ ]:
from scipy.stats import mannwhitneyu
mannwhitneyu(wind.loc[wind.wind_cat == 'med-wind', 'broad_power'].values, wind.loc[wind.wind_cat == 'high-wind', 'broad_power'].values)

: 

## Scatter of air vs buried

In [ ]:
d = pd.to_datetime('2001-02-01')
d = d.tz_localize('UTC')
t = [d + pd.Timedelta(f'{si} hours') for si in range(24)]
t_local = [t.tz_convert('America/Boise') for t in t]
# t_hour = [t.time() for t in t_local]
for i, d in enumerate(t_local):
    if d.day == 31:
        t_local[i] = t_local[i] + pd.Timedelta('1 day')

with open('/bsuhome/zacharykeskinen/infrasound/results/hourly_average/v1', 'rb') as f:
    hourly = pickle.load(f)

hour_power = pd.DataFrame(hourly, index= t_local, columns = [0.33,0.66,1,1.33,2.0])
hour_power = hour_power.sort_index()

: 

In [ ]:
canyon = pd.read_csv('/bsuscratch/zacharykeskinen/data/infrasound/snotel/canyon_wx.csv', comment = '#', parse_dates=['Date_Time'], index_col = ['Date_Time'])
units = canyon.iloc[0]
canyon = canyon.iloc[1:]
# convert all columns of DataFrame
canyon = canyon.apply(pd.to_numeric, errors = 'ignore')
canyon = canyon.loc[:pd.to_datetime('2022-05-15')]
canyon = canyon.tz_convert('UTC')

: 

In [ ]:
s = canyon.loc[(canyon.index > pd.to_datetime('2022-01-01T00:00:00 UTC')) & (canyon.index < pd.to_datetime('2022-04-01T00:00:00 UTC')), 'wind_speed_set_1']

: 

In [ ]:
import matplotlib.dates as mdates
from dateutil import tz
fig, ax = plt.subplots()
for col in hour_power.columns:
    sns.lineplot(x = hour_power.index, y = col, data = hour_power, ax = ax, label = f'{col} m')
plt.yscale('log')
plt.title('Average Power for Buried vs Free-Air Stations')
plt.ylabel('Hourly Power ($Pa^{2}$)')
plt.xlabel('Time of Day')
# plt.gca().xaxis.set_major_locator(plt.MaxNLocator(7, integer = True))
myFmt = mdates.DateFormatter('%H:%M', tz = tz.gettz('America/Boise'))

ax.xaxis.set_major_formatter(myFmt)
# ax.xaxis.set_major_locator(mdates.HourLocator())
# fig.autofmt_xdate()

: 

In [ ]:
wind

: 